In [12]:
#import libraries
import numpy as np
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [34]:
#import another ipynb file
%run  '../gusssian_linear_discriminate_analysis.ipynb'

In [29]:
#function for plot datas------------------------------
def scotplot(x,y,pred,ax):
    mis_class0=False
    mis_class1=False
    xone=x[:,0]
    xtwo=x[:,1]
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.xlim((np.min(xone)-1, np.max(xone)+1))
    plt.ylim((np.min(xtwo)-1, np.max(xtwo)+1))
    for i in range(len(y)):
        if(y[i]==pred[i] and y[i]==0 ):
            cls0=ax.scatter(xone[i], xtwo[i],  alpha=.5,marker='o',color='g')
        if(y[i]==pred[i] and y[i]==1 ):
            cls1=ax.scatter(xone[i], xtwo[i], alpha=.5, marker='o',color='b')
        if(y[i]!=pred[i] and y[i]==0 ):
            mis0=ax.scatter(xone[i], xtwo[i],  marker='x',color='g')
            mis_class0=True
        if(y[i]!=pred[i] and y[i]==1 ):
            mis1=ax.scatter(xone[i], xtwo[i],  marker='x',color='b')
            mis_class1=True

    if(mis_class0 and mis_class1):
        ax.legend([mis0,mis1], ['miss 0', 'miss 1'])

    elif(mis_class0):
        ax.legend([mis0], ['miss 0'])

    elif(mis_class1):
        ax.legend([mis1], ['miss 1'])

    return cls0,cls1

#function for load data-----------------------------
def load_data(fname):
    csv = np.genfromtxt(fname, dtype=float ,delimiter=",")
    datas = np.array(csv)
    X = np.array(datas[:,0:2])
    Y = np.array(datas[:,2])
    return X,Y

In [ ]:
#load data-----------------------------------------
fname1 = '../data/BC-Train1.csv'
fname2 = '../data/BC-Test1.csv'

X_train,Y_train = load_data(fname1)
X_test,Y_test = load_data(fname2)

#get parameters-----------------------------------------
model = glda()

model.get_len_class1(Y_train)
model.get_len_class0(Y_train)

Phi = model.get_phi(Y_train)
Mu0 = model.get_mu0(X_train,Y_train).reshape(2,1)
Mu1 = model.get_mu1(X_train,Y_train).reshape(2,1)
Sigma = model.get_covariance(X_train,Y_train,Mu0,Mu1)

results = " \nphi0:\n %s \nphi1:\n %s \nMu0:\n %s \nMu1:\n %s \nsigma:\n %s "
print(results % ( 1-Phi,Phi, Mu0, Mu1, Sigma))

#predict labels and calculate accuracy--------------------------
preds_train = model.predict(X_train,Mu0,Mu1,Sigma, Phi)
preds_test = model.predict(X_test,Mu0,Mu1,Sigma, Phi)


print("Train data:")
train_acc=model.score(preds_train,Y_train)
print("train accuracy:",train_acc*100,"%")

print("\nTest data :")
test_acc=model.score(preds_test,Y_test)
print("test accuracy:",test_acc*100,"%")

#plotting scatter data and decision boundry--------------------
fig1=plt.figure(figsize=(8,8))
fig1.suptitle('Scatter Plot and decision boundary')

ax = fig1.subplots(1, 2)
for ax1 in ax.flat:
    ax1.set(xlabel='x1', ylabel='x2')


ax[1].set_title('Train set')
ax[0].set_title('Test set')

corct0,corct1=scotplot(X_train,Y_train,preds_train,ax[1])
scotplot(X_test,Y_test,preds_test,ax[0])

model.plot_decision_boundary(X_train , Mu0, Mu1, Sigma, Phi,ax[0])
model.plot_decision_boundary(X_train , Mu0, Mu1, Sigma, Phi,ax[1])
plt.savefig("../images/Scatter Plot data1.jpg")

blackline = mlines.Line2D([], [], color='black')
fig1.legend([corct0,corct1,blackline], ['class 0', 'class 1', 'Linear Boundary'])

model.plot2d(X_train,Mu0, Mu1, Sigma,Phi)
plt.savefig("../images/2D estimated PDFs data1.jpg")

model.plot3d(X_train,Mu0, Mu1, Sigma,Phi)
plt.savefig("../images/3D estimated PDF data1.jpg")
plt.show()
